In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib
from scipy.signal import find_peaks
matplotlib.use('Qt5Agg')

In [2]:
class spek:
    def __init__(self,path):
        data=np.loadtxt(path)
        self.steps=[]
        self.intensity=[]
        self.ru_lines=[495.31,496.39,498.34,501.61,502.30,506.40,507.2,507.66,508.94,513.39,515.15,517.11,519.67,523.54,526.15,532.38,536.41,539.21,543.31,558.03,564.93,565.53,572.57,589.16,598.75,607.24,616.13,620.8,630.01]
        for line in data:
            self.steps.append(line[0])
            self.intensity.append(line[1])
        #print(self.peaks_index)
        self.steps,self.intensity=np.array(self.steps),np.array(self.intensity)
        self.peaks_index=self.peaks()[0]
        self.peaks_index=np.delete(self.peaks_index,[0,2])
        print(len(self.peaks_index),len(self.ru_lines))

    def peaks(self):
        return find_peaks(self.intensity,height=0.01)

    def plot(self):
        fig = plt.figure(figsize=(11, 6))
        gs = GridSpec(8, 5)
        fig1 = fig.add_subplot(gs[:, :])
        fig1.set_title("Sektrum")
        fig1.set_ylabel("step")
        fig1.set_xlabel("Intensity")
        fig1.set_xlim(10000,40000)
        # fig1.plot(self.steps,self.intensity,label=f"plot")
        # fig1.plot(self.steps[self.peaks_index],self.intensity[np.array(self.peaks_index)],"o",label="peaks")
        fig1.scatter(self.steps[self.peaks_index],self.ru_lines)
        plt.tight_layout()
        plt.legend()
        plt.show()

In [3]:
file="ru_spektrum.dat"#"1_-4_ohneLinse.dat"
spek1=spek(f"data/{file}")
spek1.plot()
